In [1]:
import datetime
import requests
import pandas as pd
import os
import json
# import datetime
# from datetime import datetime
from datetime import timedelta
from datetime import date
from pathlib import Path
import shutil
import time
from dotenv import dotenv_values
from selenium import webdriver
from selenium.webdriver.firefox.webdriver import WebDriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import *
from selenium.webdriver import Keys, ActionChains
import calendar
from re import search
import pandas.io.common

In [3]:
# classe para importar tabelas para o storm
    ## está tudo certo, só preciso ver porque as fuções mazzpaths e makeTempPaths estão zerando os arquivos
class workTablesToStorm():
    
    def __init__(self, datework: date):
        self.storm = False
        self.datework = datework
        self.istable()
        self.toStorm()
        self.save()
        self.makezzpaths()
        self.makeTempPaths()
        self.concatZZ()
    
    def istable(self):
        '''
        Função que libera as demais, se o arquivo com a data for encontrado.
        Define o arquivo encontrado como atributo "dados"
        '''
        path = f'./download/'
        listdir = os.listdir(path)
        for dir in listdir:
            if search(str(self.datework), dir):
                if os.path.getsize(path + dir) > 0:
                    dados = pd.read_csv(path + dir,
                            delimiter=";",
                            encoding="ISO-8859-1", decimal=".")
                    setattr(self, 'dados', dados)
                    self.storm = True
                
                            
    def toStorm(self):
        '''
        Se o arquivo for encontrado, os relatórios para o storm serão gerados.
        Os atributos banks e columns serão definidos
        '''
        
        if self.storm:
            dados = self.dados
            banks = dados.NOM_BANCO.unique()
            columns = dados.columns
            setattr(self, 'banks', banks)
            setattr(self, 'columns', columns)
            dados = dados[['NOM_BANCO', 'NUM_PROPOSTA', 'VAL_BASE_COMISSAO', 'VAL_COMISSAO_TOTAL', 'QTD_PARCELA', 'DAT_EMPRESTIMO']]
            dados.insert(4, "#VALOR_BASE_BRUTO#", dados['VAL_BASE_COMISSAO'])
            dados.columns = ['NOM_BANCO', '#ADE#', '#VALOR_BASE#', '#VALOR_CMS#', '#PRAZO#', '#VALOR_BASE_BRUTO#', '#DATA_DIGITACAO#']
            for bank in banks:
                path_to_save = f'../Importados_storm/Comissao/{self.datework.year}/{self.datework.month}/{bank}'
                os.makedirs(path_to_save, exist_ok=True)
                report = dados[dados.NOM_BANCO == bank]
                report.to_excel(excel_writer = path_to_save + f'/Comissao_{date.strftime("%d-%m-%Y")}.xlsx', index = False)
                
    def save(self):
        '''
        Depois de encontrar o arquivo, definir os atributos, está função aplica filtro por banco.
        Salva um atributo de DF com o nome do banco
        '''
        if self.storm:
            dados = self.dados
            banks = self.banks
            for bank in banks:
                dados = dados[dados.NOM_BANCO == bank]
                setattr(self, f'{bank}', dados)
                
    
    def makezzpaths(self):
        '''
        Função para criar zz_geral para cada banco, uma única vez.
        '''
        if self.storm:
            for bank in self.banks: 
                path = f'zz_geral_{bank}.xlsx'
                if path not in os.listdir("../"):
                        zz = pd.DataFrame(columns=self.columns)
                        zz.to_excel("../"+path, index=False, sheet_name = f'Producao_ate{self.datework.strftime("%Y-%m-%d")}')
                        
        
                        
    def makeTempPaths(self):
        '''
        Função faz uma cópia do zz_geral para o tmp
        '''
        if self.storm:
            os.makedirs("tmp", exist_ok=True)
            for bank in self.banks: 
                path = f'../zz_geral_{bank}.xlsx'
                tempPath = f'tmp/zz_geral_{bank}_temp.xlsx'
                shutil.copy(path, tempPath)
                
                
    def concatZZ(self):
        '''
        função lê o arquivo temporáriio, captura os dados filtrados pelo NOM_BANK da função save
        se tiver arquivo para a data, depois de ser copiado para o temp
        - lê o temp
        - filtra por data
        - concatena novos dados do banco
        - sobscreve o temp com os novos dados
        - faz o novo zz geral
        
        '''
        
        if self.storm:
            dados = self.dados
            print(self.datework)
            
            for bank in self.banks:
                print(bank)
                print(dados.NOM_BANCO.unique())
                
                zzpath = pd.read_excel(f'tmp/zz_geral_{bank}_temp.xlsx')
                if self.datework not in zzpath['DAT_EMPRESTIMO']:
                    print("data não encontrada")
                    dados_to_save = dados[dados.NOM_BANCO == bank]
                    print(dados_to_save)
                    concatenado = pd.concat([zzpath, dados_to_save])
                    concatenado.drop_duplicates(inplace=True)
                    concatenado.to_excel(excel_writer = f'tmp/zx_geral_{bank}_temp.xlsx', 
                                        index = False, 
                                        sheet_name = f'Producao_ate{self.datework.strftime("%Y-%m-%d")}')
                    shutil.move(f'tmp/zx_geral_{bank}_temp.xlsx', f'../zz_geral_{bank}.xlsx')
                print("Data encontrada")
                
        
from datetime import date
date = date.today() - timedelta(days=90)
while date <  date.today(): 
    workTablesToStorm(date)
    date = date + timedelta(days = 1)
    # print(date)            
            
   

2023-08-14
BANCO PAN
['BANCO PAN']
data não encontrada
    NUM_BANCO  NOM_BANCO  NUM_PROPOSTA  NUM_CONTRATO  \
0         623  BANCO PAN     776628480     776628480   
1         623  BANCO PAN     176406758     176406758   
2         623  BANCO PAN     176500681     176500681   
3         623  BANCO PAN     176518051     176518051   
4         623  BANCO PAN     176528736     176528736   
5         623  BANCO PAN     176539452     176539452   
6         623  BANCO PAN     176545754     176545754   
7         623  BANCO PAN     176545920     176545920   
8         623  BANCO PAN     176549192     176549192   
9         623  BANCO PAN     176550888     176550888   
10        623  BANCO PAN     176550900     176550900   
11        623  BANCO PAN     176555894     176555894   
12        623  BANCO PAN     176557263     176557263   
13        623  BANCO PAN     176559415     176559415   
14        623  BANCO PAN     176568837     176568837   
15        623  BANCO PAN     176571335     176571

In [45]:
from datetime import date
date = date(2023, 10, 31)
while date <  date.today(): 
    workTablesToStorm(date)
    date = date + timedelta(days = 1)
    print(date)

BANCO PAN
data não encontrada
     NUM_BANCO  NOM_BANCO  NUM_PROPOSTA  NUM_CONTRATO  \
0          623  BANCO PAN     378786971     378786971   
1          623  BANCO PAN     779403086     779403086   
2          623  BANCO PAN     379513370     379513370   
3          623  BANCO PAN     179553516     179553516   
4          623  BANCO PAN     379574157     379574157   
..         ...        ...           ...           ...   
374        623  BANCO PAN     379773423     379773423   
375        623  BANCO PAN     379775570     379775570   
376        623  BANCO PAN     379778110     379778110   
377        623  BANCO PAN     379786885     379786885   
378        623  BANCO PAN     379788513     379788513   

                           NOM_CLIENTE  COD_CPF_CLIENTE  \
0    MARIA RAIMUNDA SANTOS DE CARVALHO      31595138587   
1             GIULIA DOS SANTOS FREIRE      15828411799   
2                        REINALDO DIAS         43262708   
3           FRANCISCO  DE  SOUSA NANJA       1331